In [98]:
import pandas as pd

In [99]:
def create_timesbasert_dataframe(path_med: str, path_kir: str) -> pd.DataFrame:
    hf_med_time = pd.read_excel(path_med, sheet_name="Results")
    hf_kir_time = pd.read_excel(path_kir, sheet_name="Results")

    hf_med_time = hf_med_time[hf_med_time["Timer"].apply(lambda x: not (pd.isna(x) or x == "."))]
    hf_med_time.reset_index(inplace=True)
    hf_med_time.drop("index", axis=1, inplace=True)

    hf_kir_time = hf_kir_time[hf_kir_time["Timer"].apply(lambda x: not (pd.isna(x) or x == "."))]
    hf_kir_time.reset_index(inplace=True)
    hf_kir_time.drop("index", axis=1, inplace=True)

    hf_med_time["post"] = "medisinsk"
    hf_kir_time["post"] = "kirurgisk"


    fin_data = pd.concat([hf_med_time, hf_kir_time], axis=0).sort_values("Dato").reset_index()
    fin_data.drop(["index"], axis=1, inplace=True)
    fin_data['helg'] = fin_data['Dato'].dt.weekday.apply(lambda x: 1 if x >= 5 else 0)

    fin_data["År"] = fin_data['Dato'].dt.year
    fin_data["Måned"] = fin_data['Dato'].dt.month_name()

    fin_data = fin_data[["År", "Måned", "Dato", "Timer", "post", "helg", "Antall inn på post", "Antall pasienter ut av Post"]]
    fin_data.sort_values(by=["Dato", "Timer"])
    fin_data["Timer"] = fin_data["Timer"].astype(int) 

    return fin_data


def calculate_patients(row):
    global current_patients
    current_patients += row["Antall inn på post"] - row["Antall pasienter ut av Post"]
    current_patients = max(0, current_patients)
    
    return current_patients


def add_shift_type(row):
    if 6 < row["Timer"] <= 14:
        shift_type = "dag"
    elif 14 < row["Timer"] <= 22:
        shift_type = "kveld"
    else: 
        shift_type = "natt"
        
    return shift_type

In [100]:
fin_hf_med_path = "./data/med_pasientstrøm_time_2024.xlsx"
fin_hf_kir_path = "./data/kir_pasientstrøm_time_2024.xlsx"

fin_data = create_timesbasert_dataframe(fin_hf_med_path, fin_hf_kir_path)

In [101]:
initial_patients = 0
current_patients = initial_patients

fin_data["Belegg"] = fin_data.apply(calculate_patients, axis=1)
fin_data["skift_type"] = fin_data.apply(add_shift_type, axis=1)

In [102]:
fin_data

,År,Måned,Dato,Timer,post,helg,Antall inn på post,Antall pasienter ut av Post,Belegg,skift_type
0,2024,January,2024-01-01,0,medisinsk,0,2.0,0.0,2.0,natt
1,2024,January,2024-01-01,2,medisinsk,0,0.0,0.0,2.0,natt
2,2024,January,2024-01-01,3,medisinsk,0,1.0,0.0,3.0,natt
3,2024,January,2024-01-01,4,medisinsk,0,1.0,0.0,4.0,natt
4,2024,January,2024-01-01,5,medisinsk,0,0.0,0.0,4.0,natt
...,...,...,...,...,...,...,...,...,...,...
13819,2024,October,2024-10-14,5,kirurgisk,0,0.0,0.0,21.0,natt
13820,2024,October,2024-10-14,4,kirurgisk,0,0.0,0.0,21.0,natt
13821,2024,October,2024-10-14,3,kirurgisk,0,0.0,0.0,21.0,natt
13822,2024,October,2024-10-14,13,kirurgisk,0,1.0,0.0,22.0,dag
